<a href="https://colab.research.google.com/github/RodolfoMontes/SmarttBot-Spark/blob/main/Sarmttbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

## [SparkSession](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.html)

O ponto de entrada para programar o Spark com a API Dataset e DataFrame.

Uma SparkSession pode ser utilizada para criar DataFrames, registrar DataFrames como tabelas, executar consultas SQL em tabelas, armazenar em cache e ler arquivos parquet. Para criar uma SparkSession, use o seguinte padrão de construtor:

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [ ]:
spark

[Site ngrok](https://ngrok.com)

In [ ]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [ ]:
spark

### Montando nosso drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Carregando os dados

In [ ]:
person_person = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Person_Person.csv', sep=';', inferSchema=True)
production_product = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Production_Product.csv', sep=';', inferSchema=True)
sales_customer = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Sales_Customer.csv', sep=';', inferSchema=True)
sales_salesorderdetail = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Sales_SalesOrderDetail.csv', sep=';', inferSchema=True)
sales_salesorderheader = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Sales_SalesOrderHeader.csv', sep=';', inferSchema=True)
sales_specialofferproduct = spark.read.options(header=True).csv('/content/drive/MyDrive/SmarttBot-Spark/Sales_SpecialOfferProduct.csv', sep=';', inferSchema=True)

##1 - Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail pelo campo SalesOrderID.

In [ ]:
sales_salesorderdetail.createOrReplaceTempView("sales_salesorderdetail")

In [ ]:
spark.sql("select count(SalesOrderID) as SalesOrderID from sales_salesorderdetail").show()

+------------+
|SalesOrderID|
+------------+
|      121317|
+------------+



In [ ]:
sales_salesorderdetail\
  .select('SalesOrderID').alias('SalesOrderID')\
  .count()

121317

## 2 - Escreva uma query que ligue as tabelas Sales.SalesOrderDetail, Sales.SpecialOfferProduct e Production.Product e retorne os 3 produtos (Name) mais vendidos (pela soma de OrderQty), agrupados pelo número de dias para manufatura (DaysToManufacture)

In [ ]:
production_product_join = production_product\
  .join(sales_salesorderdetail, 'productid', how='inner')\
  .join(sales_specialofferproduct, 'productid', how='inner')\

production_product_join.createOrReplaceTempView("a")
spark.sql("SELECT Name,  DaysToManufacture, sum(OrderQty) as OrderQty from a group by 1, 2  ORDER BY 3 DESC LIMIT 3 ").toPandas()

,Name,DaysToManufacture,OrderQty
0,"Sport-100 Helmet, Blue",0,33715
1,AWC Logo Cap,0,33244
2,"Sport-100 Helmet, Black",0,32660
